In [162]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

from skimage.filters import threshold_otsu
from skimage.morphology import closing, square

In [163]:
video_path = "../video/rilevamento-intrusioni-video.wm"
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), "Not opened!"

fps = int(cap.get(cv2.CAP_PROP_FPS))
length = cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps

print(f"[I] Video FPS: {fps}")
print(f"[I] Video Length: {length}")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"[I] Video Frame Width: {width}")
print(f"[I] Video Frame height: {height}")

[I] Video FPS: 12
[I] Video Length: 41.916666666666664
[I] Video Frame Width: 320
[I] Video Frame height: 240


In [164]:
# Trying to understand MOG2 parameters

# # BackgroundRatio --> ok default (0.9) per ora quindi
# If a foreground pixel keeps semi-constant value for about backgroundRatio*history frames, it's considered background and added to the model 
# as a center of a new component.
# "In altre parole, TB controlla quanto velocemente il modello di sfondo si adatta ai cambiamenti nella scena."
# - Un valore più alto rende il modello di sfondo più adattabile e meno sensibile ai movimenti.
# - Un valore più basso rende il modello di sfondo più stabile e più sensibile ai movimenti, ma anche più lento ad adattarsi.

# # VarThreshold

In [165]:
# MOG2 Background Subtractor
mog2 = cv2.createBackgroundSubtractorMOG2()
mog2_test = cv2.createBackgroundSubtractorMOG2()

mog2.setHistory(fps*6)
mog2.setDetectShadows(False)
mog2.setVarThreshold(9)
mog2.setBackgroundRatio(0.6)

mog2_test.setHistory(fps)
mog2_test.setDetectShadows(False)
mog2_test.setVarThreshold(9)
mog2_test.setBackgroundRatio(0.6)

In [166]:
# """
#     Performed a sorted insertion of pv into the frame buffer in the pi row.
#     pv: pixel value
#     pi: pixel index
#     frame_buffer (global)
# """
# def sortAndInsert(pv, pi, ):
#     global frame_buffer
#     insertion_idx = np.searchsorted(frame_buffer[pi], pv)
#     if insertion_idx == len(frame_buffer[pi]):
#         insertion_idx =- 1

#     frame_buffer[pi, insertion_idx] = pv

#cv2.imshow("frame_copy", frame_contours)
# largestContour = max(contours, key = cv2.contourArea) # get largest contour
# rect = cv2.minAreaRect(largestContour)
# box = np.int64(cv2.boxPoints(rect))
# cv2.drawContours(frame_contours, [box], 0, (0,0,255), 1)
# cv2.imshow("frame_contours", frame_contours)

# cv2.imshow(f"temporalMedianBackground-frame-{frame_count}", temporalMedianBackground)
# cmd = cv2.waitKey(0)
# cv2.destroyWindow(f"temporalMedianBackground-frame-{frame_count}")
# if cmd == ord("q"):
#     break
# if cmd == ord("n"):
#     continue

# alpha = 3.0 # Contrast control
# beta = 25 # Brightness control
# # call convertScaleAbs function
# #adjusted = cv2.convertScaleAbs(mog2_fgmask, alpha=alpha, beta=beta)


# plt.hist(frame_diff.ravel(), 256, [0, 256])
# plt.show()


            
# Disegna la bounding box
#cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)

#cv2.imshow("frame_contours", frame_contours)


# analysis = cv2.connectedComponentsWithStats(tmb_fgmask, 8, cv2.CV_32S)
# (totalLabels, label_ids, values, centroid) = analysis 

# # Initialize a new image to store  
# # all the output components 
# output = np.zeros(tmb_fgmask.shape, dtype="uint8") 

# # Loop through each component 
# for i in range(1, totalLabels): 
    
#     # Area of the component 
#     area = values[i, cv2.CC_STAT_AREA]  
    
#     if (area > 140) and (area < 400): 
#         componentMask = (label_ids == i).astype("uint8") * 255
#         output = cv2.bitwise_or(output, componentMask) 

# cv2.imshow("Filtered Components", output)
# 
# 
#mog2 = cv2.createBackgroundSubtractorMOG2(history=fps*2, varThreshold=20, detectShadows=False)

# print(mog2.getHistory())
# print(mog2.getVarThreshold())
# print(mog2.getBackgroundRatio())
# print(mog2.getNMixtures())
# print(mog2.getVarThresholdGen())
# print(mog2.getVarMin())
# print(mog2.getVarMax())
# print(mog2.getComplexityReductionThreshold())
# print(mog2.getVarInit())

#mog2.setComplexityReductionThreshold(0)
#mog2.setBackgroundRatio(0.7)
#mog2.setNMixtures(1)
#mog2.setVarThresholdGen(30) 

#temporalMedianBackground_tmp = np.median(frames_tmp, axis=0).astype(dtype=np.uint8)
#cv2.imshow("temporal background - true one", temporalMedianBackground)
# make addWeighted between temporalMedianBackground and frame just where the background_mask is 1
#masked_img = np.where(background_mask[..., None] == 255, frame[..., None], 0) # dove c'è il background mask metti frame, altrimenti 0
#cv2.imshow("masked_img", masked_img)
#temporalMedianBackground = cv2.addWeighted(temporalMedianBackground, 0.7, masked_img, 0.3, 0)
#temporalMedianBackground = cv2.bitwise_and(temporalMedianBackground, temporalMedianBackground, background_mask)



In [167]:
def preprocessing(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply Median Blur
    median = cv2.medianBlur(gray, 5)
    gaussian = cv2.GaussianBlur(median, (7, 7), 3)
    # Apply Bilateral Filter
    filtered = cv2.bilateralFilter(gaussian, 9, 75, 75)
    return filtered

In [168]:
# HYPERPARAMTERS
tuning_history = 6
tuning_skip_frame = 4
MAX_HISTORY = tuning_history * fps
SKIP_FRAMES = fps // tuning_skip_frame
THRESHOLD = 30

# 320x240 rows, 60 columns
# frame_buffer = np.empty((width * height,MAX_HISTORY), dtype=np.uint8)

start = time.time()
frames = []
initialized = False
frame_count = 0
skip_count = 0
fgmask_diff = None
temporalMedianBackground = None
ratios = []

kernel = np.ones((3,3),np.uint8)

while(cap.isOpened()):
    ret, frame_original = cap.read()
    if not ret or frame_original is None:
        cap.release()
        print("Released Video Resource")
        break

    frame = preprocessing(frame_original)
    #cv2.imshow("frame", frame)
    frame_count += 1
    skip_count += 1

    history = mog2.getHistory()
    # history_test = mog2_test.getHistory()
    
    if frame_count < history:
        learning_rate = 1 / frame_count
    else:
        learning_rate = 1 / history
  
    learning_rate = learning_rate + 0.1
    # learning_rate_test = learning_rate_test + 0.1
    mog2_fgmask = mog2.apply(frame, learningRate=learning_rate)
    erode = cv2.erode(mog2_fgmask, None, iterations=1)
    dilated = cv2.dilate(erode, None, iterations=3)
    mog2_fgmask = dilated.copy()
    #cv2.imshow("Mog2 foreground mask", mog2_fgmask)
    #cv2.imshow("Eroded mog2 foreground mask", dilated)
   
    # # tentativo
    blur = cv2.GaussianBlur(mog2_fgmask, (5,5), 3)
    thresh = threshold_otsu(blur)
    if thresh > 0:    
        bw = closing(blur >= thresh, square(7))
        #erode = cv2.erode(bw.astype(np.uint8) * 255, None, iterations=3)
        #cv2.imshow("erode", erode)
        erode = cv2.medianBlur(bw.astype(np.uint8) * 255, 3)
        dilated = cv2.dilate(erode, None, iterations=5)
        #cv2.imshow("dilate", dilated)        

        frame_contours = frame.copy()
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        real_countours = []
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            
            if cv2.contourArea(contour) > 100:
                real_countours.append(contour)
            
        # evitare che impazzisca se non ci sono bboxes
        if len(real_countours) > 0:
            # Calcola la bounding box
            # Define the ROI            
            x_min = min([np.min(cnt[:, :, 0]) for cnt in real_countours])
            x_max = max([np.max(cnt[:, :, 0]) for cnt in real_countours])
            y_min = min([np.min(cnt[:, :, 1]) for cnt in real_countours])
            y_max = max([np.max(cnt[:, :, 1]) for cnt in real_countours])

            # Disegna la bounding box
            #cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)

        #cv2.imshow("frame_contours", frame_contours)


    if initialized:
        # print(f"[I] Skipping Frames... {skip_count}")
        if skip_count > SKIP_FRAMES:
            skip_count = 0
            frames.pop(0)
            frames.append(frame)
            temporalMedianBackground = np.median(frames, axis=0).astype(dtype=np.uint8)
            #cv2.imshow("temporal background", temporalMedianBackground)
            
    if not initialized and len(frames) < MAX_HISTORY:
        # print(f"[I] Learning...")
        frames.append(frame)
        
    elif not initialized and len(frames) == MAX_HISTORY:
        # print(f"Compute First Estimate")
        # compute the first background estimation
        initialized = True    
        temporalMedianBackground = np.median(frames, axis=0).astype(dtype=np.uint8)

    # Compute the AbsDiff between temporalMedianBackground and the Current Frame
    if initialized:
        frame_diff = cv2.absdiff(temporalMedianBackground, frame)
        threshold, tmb_fgmask = cv2.threshold(frame_diff, THRESHOLD, 255, cv2.THRESH_BINARY)
        #cv2.imshow("tmb_fgmask", tmb_fgmask)
        
        # logical or between mog2 and temporalMedianBackground to create the shape of the fast movement using ROI
        fgmask = mog2_fgmask.copy()
        fgmask[y_min:y_max, x_min:x_max] = cv2.bitwise_or(mog2_fgmask[y_min:y_max, x_min:x_max], tmb_fgmask[y_min:y_max, x_min:x_max])    
        #cv2.imshow("Bitwise Mask", fgmask)
    
        closing_img = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel, iterations=3)
        #cv2.imshow("closing_img", closing_img)
        
        count_white = cv2.countNonZero(closing_img)
        total_pixels = width * height
        rapp = (total_pixels - count_white) / total_pixels
        ratios.append(rapp)
        #print(f"Background ratio: {rapp}")

        #cv2.imshow("background mask mog2", background_mask)

        # override frozen values
        if fgmask_diff is not None:
            tmb_fgmask = cv2.bitwise_or(tmb_fgmask, fgmask_diff)
            #tmb_fgmask = cv2.morphologyEx(tmb_fgmask, cv2.MORPH_CLOSE, kernel, iterations=3)
            cv2.imshow("tmb_fgmask", tmb_fgmask)
            # TODO: contours and areas

        # static objects
        fgmask_diff = cv2.bitwise_and(tmb_fgmask, cv2.bitwise_not(closing_img))
        #fgmask_diff = cv2.medianBlur(fgmask_diff, 3)
        #cv2.imshow("fgmask_diff", fgmask_diff)
        #fgmask_diff = cv2.dilate(fgmask_diff, None, iterations=3)

        #cv2.imshow("fgmask_diff", fgmask_diff)

    cmd = cv2.waitKey(0)    
    if cmd == ord("q"):
        break
    if cmd == ord("n"):
        continue
print("Mean Background ratio: ", np.mean(ratios))
# valore simile a quelo di default --> lasciamo quello di default

end = time.time()

print(f"[I] Elapsed time: {end - start} s")

cap.release()
cv2.destroyAllWindows()


/tmp/ipykernel_56242/3128952899.py:55: FutureWarning: `square` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  bw = closing(blur >= thresh, square(7))


Released Video Resource
Mean Background ratio:  0.8507272029679042
[I] Elapsed time: 24.45100998878479 s
